### Question 1. Understanding docker first run

Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image? ```Answer: 24.3.1```

### Question 2. Understanding Docker networking and docker-compose

Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

```bash
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

```Answer: db:5432 or postgres:5432```

### Question 3. Trip Segmentation Count

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

  1. Up to 1 mile
  2. In between 1 (exclusive) and 3 miles (inclusive),
  3. In between 3 (exclusive) and 7 miles (inclusive),
  4. In between 7 (exclusive) and 10 miles (inclusive),
  5. Over 10 miles


```
select
    case
        when trip_distance <= 1 then 'Up to 1 mile'
        when trip_distance > 1 and trip_distance <= 3 then '1~3 miles'
        when trip_distance > 3 and trip_distance <= 7 then '3~7 miles'
        when trip_distance > 7 and trip_distance <= 10 then '7~10 miles'
        else '10+ miles'
    end as segment,
	count(1) as num_trips,
from
    green_taxi_trips
where
    lpep_pickup_datetime >= '2019-10-01'
    and lpep_pickup_datetime < '2019-11-01'
    and lpep_dropoff_datetime >= '2019-10-01'
    and lpep_dropoff_datetime < '2019-11-01'
group by
    segment
```

### Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

```
  select
    cast(lpep_pickup_datetime as Date) as pickup_day,
    max(trip_distance) as max_trip_distance
  from
    green_taxi_trips
  group by pickup_day
  order by max(trip_distance) DESC
  limit 1

```

### Question 5. Three biggest pickup zones
Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only ```lpep_pickup_datetime``` when filtering by date.

```
  select
    CONCAT("Borough", ' / ', "Zone") as area,
    sum(total_amount) as total
  from
    green_taxi_trips,
    zones z
  where
    lpep_pickup_datetime::date = '2019-10-18'
    AND z."LocationID" = "PULocationID"
  group by
    area
  having
    sum(total_amount) > 13000
  order by total desc
```


### Question 6. Largest tip
For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's tip , not trip

We need the name of the zone, not the ID.

```
  select
    puz."Zone" as pickup_zone,
    doz."Zone" as dropoff_zone,
    g.tip_amount
  from
      green_taxi_trips g
  inner join
      zones puz on g."PULocationID" = puz."LocationID"
  inner join
      zones doz on g."DOLocationID" = doz."LocationID"
  where
      puz."Zone" = 'East Harlem North'
  order by
      g.tip_amount desc
  limit 1
```

### Question 7. Terraform Workflow
Which of the following sequences, respectively, describes the workflow for:

  1. Downloading the provider plugins and setting up backend,
  2. Generating proposed changes and auto-executing the plan
  3. Remove all resources managed by terraform`
